In [17]:
import boto3
import importlib
import utils  
importlib.reload(utils)

<module 'utils' from '/Users/i589142/Documents/GitHub/cloud-computing/utils.py'>

In [18]:
REGION = "eu-north-1"  
KEY_NAME = "test-keypair"
MY_PUBLIC_IP = '193.16.224.5/32'
KEY_NAME = "test-keypair"
AMI_UBUNTU = "ami-042b4708b1d05f512"
INSTANCE_TYPE = "t3.micro"

In [19]:
ec2_client = boto3.client("ec2", region_name=REGION)
ec2_resource = boto3.resource("ec2", region_name=REGION)

In [20]:
key_file_path = utils.create_key_pair(ec2_client, key_name=KEY_NAME)

Key-Pair test-keypair wurde erstellt und gespeichert.


In [21]:
print(key_file_path)

test-keypair.pem


In [22]:
security_group_id = utils.create_security_group(ec2_client, group_name="main-security-group", my_public_ip=MY_PUBLIC_IP, desc="My decription.", ssh=True, http=True, flask=True)

Created security group sg-0f259fed06d1c90ae
Security Group ingres authorized.


In [23]:
instance = utils.create_instance(ec2_resource, instance_name="test-vm", image_id=AMI_UBUNTU, instance_type=INSTANCE_TYPE, key_name=KEY_NAME, security_group_id=security_group_id)

Instanz-ID: i-033bb05faa2695af7 wird gestartet...
Instanz läuft. Öffentliche IP: 16.170.203.160


In [24]:
!chmod 400 {key_file_path}

In [25]:
!ssh-keyscan {instance.public_ip_address} >> ~/.ssh/known_hosts

In [27]:
!ansible-playbook install_flask_app.yml -i {instance.public_ip_address}, -u ubuntu --private-key {key_file_path}


PLAY [Deploy Flask app in home directory] **************************************

TASK [Gathering Facts] *********************************************************
[WARNING]: Platform linux on host 16.170.203.160 is using the discovered Python
interpreter at /usr/bin/python3.12, but future installation of another Python
interpreter could change the meaning of that path. See
https://docs.ansible.com/ansible-
core/2.18/reference_appendices/interpreter_discovery.html for more information.
ok: [16.170.203.160]

TASK [Update apt cache] ********************************************************
changed: [16.170.203.160]

TASK [Install system packages] *************************************************
changed: [16.170.203.160]

TASK [Clone Flask app into home directory] *************************************
changed: [16.170.203.160]

TASK [Start Flask app with nohup] **********************************************
changed: [16.170.203.160]

PLAY RECAP ********************************************

In [28]:
instance.terminate()

{'TerminatingInstances': [{'InstanceId': 'i-033bb05faa2695af7',
   'CurrentState': {'Code': 32, 'Name': 'shutting-down'},
   'PreviousState': {'Code': 16, 'Name': 'running'}}],
 'ResponseMetadata': {'RequestId': 'fae76587-9e3b-4a6e-b544-ac1df69887d1',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': 'fae76587-9e3b-4a6e-b544-ac1df69887d1',
   'cache-control': 'no-cache, no-store',
   'strict-transport-security': 'max-age=31536000; includeSubDomains',
   'content-type': 'text/xml;charset=UTF-8',
   'content-length': '426',
   'date': 'Sat, 28 Jun 2025 09:43:23 GMT',
   'server': 'AmazonEC2'},
  'RetryAttempts': 0}}

In [ ]:
ec2_client.delete_security_group(GroupId=security_group_id)

In [ ]:
ec2_client.delete_key_pair(KeyName=KEY_NAME)